In [0]:
import pandas as pd
import numpy as np
!pip install python-constraint
import constraint

  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24080 sha256=4eddacc33c5a049eb7a9dc7e2f9f32593b9c1fb5212f624a8591eafd34be5d40
  Stored in directory: /root/.cache/pip/wheels/34/31/15/7b070b25d0a549d20ce2e9fe6d727471c2c61ef904720fd40c
Successfully built python-constraint


In [0]:
# external data we need
test = pd.read_csv("/content/gdrive/My Drive/ML_APPROACH/Data/1percent/1_test_set.csv")
y_test = test.hotel_cluster

# our predicted probabilities
proba_NB = pd.read_csv("pred_proba_NB.csv")
proba_RF = pd.read_csv("pred_proba_RF.csv")
proba_XGB = pd.read_csv("pred_proba_XGB.csv")
proba_MLP = pd.read_csv("pred_proba_MLP.csv")

In [0]:
# check shapes are okay
print(y_test.shape)
print(proba_NB.shape)
print(proba_RF.shape)
print(proba_XGB.shape)
print(proba_MLP.shape)

(74925,)
(74925, 100)
(74925, 100)
(74925, 100)
(74925, 100)


In [0]:
list_NB = []
list_RF = []
list_XGB = []
list_MLP = []
for i in range(0,100):
    list_NB.append("NB"+str(i))
    list_RF.append("RF"+str(i))
    list_XGB.append("XGB"+str(i))
    list_MLP.append("MLP"+str(i))

proba_NB.columns = list_NB
proba_RF.columns = list_RF
proba_XGB.columns = list_XGB
proba_MLP.columns = list_MLP

In [0]:
frames = [proba_NB, proba_RF, proba_XGB, proba_MLP,y_test]
data = pd.concat(frames, axis=1)
data.head()

,NB0,NB1,NB2,NB3,NB4,NB5,NB6,NB7,NB8,NB9,NB10,NB11,NB12,NB13,NB14,NB15,NB16,NB17,NB18,NB19,NB20,NB21,NB22,NB23,NB24,NB25,NB26,NB27,NB28,NB29,NB30,NB31,NB32,NB33,NB34,NB35,NB36,NB37,NB38,NB39,...,MLP61,MLP62,MLP63,MLP64,MLP65,MLP66,MLP67,MLP68,MLP69,MLP70,MLP71,MLP72,MLP73,MLP74,MLP75,MLP76,MLP77,MLP78,MLP79,MLP80,MLP81,MLP82,MLP83,MLP84,MLP85,MLP86,MLP87,MLP88,MLP89,MLP90,MLP91,MLP92,MLP93,MLP94,MLP95,MLP96,MLP97,MLP98,MLP99,hotel_cluster
0,6.957559e-11,5.225003e-10,3.508266e-09,1.907017e-11,7.018318e-09,2.896610e-05,5.717786e-10,2.963283e-13,1.897859e-10,8.756939e-08,5.262274e-07,4.441261e-10,6.020376e-19,7.068417e-10,1.192944e-14,9.484593e-12,3.190733e-08,1.201980e-07,3.827090e-06,1.070378e-10,3.063963e-10,5.171745e-05,4.020496e-13,7.248169e-10,8.814708e-18,1.409191e-07,3.578686e-10,1.291828e-21,2.413232e-07,9.107127e-12,1.213697e-08,3.270299e-13,2.340983e-12,1.266276e-07,4.067335e-13,1.123749e-16,4.022615e-13,3.881930e-06,2.816055e-12,8.166914e-12,...,7.152504e-09,2.879297e-10,5.065209e-11,0.000015,4.549240e-11,2.184690e-11,1.165742e-14,0.006671,0.030240,0.084014,3.824987e-12,0.003452,0.000096,1.481294e-08,0.000107,0.000285,0.000453,9.536412e-09,9.336023e-14,3.381601e-09,6.221819e-12,5.450110e-11,0.005562,4.135886e-07,2.005449e-08,0.001223,0.000002,4.773857e-12,0.000290,0.000004,0.000102,1.785269e-07,0.000176,0.000011,0.000276,0.000001,0.012283,0.103853,0.000020,55
1,4.955232e-11,8.778933e-09,4.590031e-10,4.656986e-16,1.618516e-09,3.659567e-07,6.162599e-09,8.931296e-12,2.479457e-14,2.129051e-09,9.987271e-10,1.760966e-12,4.063361e-19,5.012301e-10,1.053185e-15,3.502164e-11,1.004766e-06,5.700993e-11,1.192543e-06,3.167209e-11,3.104878e-14,3.968491e-08,3.660650e-15,3.942008e-12,1.124148e-15,1.200196e-08,1.172138e-10,9.481978e-26,8.911392e-07,1.047651e-14,2.234132e-10,3.543595e-14,4.776888e-10,7.720345e-08,3.793552e-11,1.544848e-20,1.166745e-14,1.293078e-08,2.550452e-16,6.799694e-09,...,2.712125e-03,1.259999e-03,1.063662e-02,0.000631,1.003014e-03,2.572854e-04,1.672909e-04,0.004558,0.000992,0.000783,1.401421e-02,0.013956,0.014582,2.989873e-04,0.003127,0.016128,0.017570,1.692374e-03,1.868592e-03,2.783609e-03,8.631475e-04,1.037793e-03,0.012563,1.960065e-02,2.970683e-03,0.000746,0.002431,2.183047e-05,0.007438,0.025696,0.060761,7.079455e-03,0.012916,0.022519,0.003496,0.053008,0.000555,0.000412,0.008289,76
2,6.832863e-10,4.703141e-08,1.399388e-10,7.880413e-15,7.162339e-10,1.669688e-07,7.946108e-09,6.182484e-13,5.645199e-13,4.517274e-09,3.699241e-09,5.899061e-13,5.053766e-17,1.682315e-10,3.254820e-15,1.723543e-11,8.454827e-08,1.501292e-09,1.735015e-06,1.447001e-13,1.717885e-13,5.723664e-07,3.144137e-14,1.127345e-11,3.389891e-15,3.292370e-08,3.240486e-10,2.581313e-23,7.399255e-08,1.548826e-13,4.392996e-10,6.434988e-13,1.623282e-11,7.025434e-08,9.541598e-12,2.345888e-19,6.089838e-14,1.342412e-08,2.721258e-15,1.023904e-09,...,1.138126e-03,4.340133e-04,1.561835e-03,0.001514,5.500597e-05,2.974617e-05,1.707983e-05,0.014852,0.004312,0.004526,1.440436e-03,0.018021,0.005897,3.324869e-04,0.002651,0.012612,0.021907,5.355195e-04,7.935529e-04,2.340901e-04,2.865102e-04,4.215999e-04,0.014838,4.335121e-03,1.894403e-03,0.000712,0.000561,4.762028e-05,0.005571,0.009269,0.073595,2.420754e-03,0.005853,0.030240,0.036728,0.011576,0.002385,0.003917,0.011738,91
3,1.402787e-13,1.058062e-14,1.022529e-06,4.511170e-13,1.227473e-11,2.488965e-03,8.929473e-09,8.259544e-12,1.018713e-06,3.018026e-07,4.168147e-07,4.740043e-08,1.210442e-18,3.510182e-11,3.011458e-12,7.260253e-10,1.154232e-07,4.606472e-08,1.877118e-09,1.181234e-16,1.467837e-09,5.971217e-06,6.522965e-08,7.771077e-14,1.349903e-24,3.293719e-04,5.562550e-11,4.082578e-21,1.595482e-06,2.824460e-09,6.559350e-06,4.242872e-13,3.084907e-13,8.787596e-09,1.258971e-14,1.406802e-14,1.239200e-11,1.535875e-05,3.087149e-10,6.140071e-16,...,9.295737e-03,7.286955e-03,1.068414e-03,0.066430,1.154560e-03,1.329263e-03,2.740775e-03,0.011961,0.009026,0.008110,4.226672e-05,0.009712,0.008664,6.660036e-05,0.016099,0.010683,0.004517,1.

In [0]:
def get_map5(w):
    # combined probabilities = probNB*w1 + probRF*w2 + probXGB*w3 + probMLP*w4
    combined_probas = (proba_NB*w[0] + proba_RF*w[1] + proba_XGB*w[2] + proba_MLP*w[3]).to_numpy() # since res would be a dataframe and map5eva doesn't accept dataframes
    map5 = map5eva(combined_probas, y_test)
    return map5

In [0]:
# GET ALL POSSIBLE VALUE COMBINATIOS FOR W=[w1, w2, w3, w4] s.t w1+w2+w3+w4 = 1
# 100 possible values for each wi (from 0.0 to 1.0 by 0.01)
ranges = np.arange(0.0, 1.1, 0.1)

problem = constraint.Problem()

problem.addVariable('w1', ranges)
problem.addVariable('w2', ranges)
problem.addVariable('w3', ranges)
problem.addVariable('w4', ranges)
# problem.addVariable('y', range(10))

def our_constraint(w1, w2, w3, w4):
    if (w1 + w2 + w3 + w4) == 1:
        return True

problem.addConstraint(our_constraint, ['w1','w2', 'w3', 'w4'])

solutions = problem.getSolutions()

In [3]:
combinations = pd.DataFrame(solutions) # combinations (saved in folder Predicted_Probabilities)

In [4]:
# add a colum to combinations with the associated map5
weights_names = ["w1", "w2", "w3", "w4"]
total = len(combinations)
for i in range(0, len(combinations)):
    combinations.loc[i, "Map5"] = get_map5(combinations.loc[i, weights_names].to_numpy())
    total = total - 1
    print(total, " combinations left")

In [0]:
# get the max map5 and the associated weights
max_map5 = max(combinations["Map5"])
index_w = combinations[combinations["Map5"] == max_map5].index

print("MAX MAP5 => ", max_map5)
print("ASSOCIATED WEIGHTS => ", combinations.loc[index_w[0], weights_names].to_numpy())

MAX MAP5 =>  0.20111378044711378
ASSOCIATED WEIGHTS =>  [0.  0.  0.3 0.7]
